In [2]:
%reload_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

### Retrieve Data

In [3]:
from stockMarket.core import get_tickers_from_index

tickers = get_tickers_from_index('sp500')

In [4]:
from stockMarket.api import populate_contracts

contracts = populate_contracts(tickers)
# contracts = populate_contracts(tickers, update_period="now")
import numpy as np
import pandas as pd

from stockMarket.core import Screener, LimitScreenerObject, EqualityScreenerObject
from stockMarket.core import growth_total

screener = Screener(contracts)

bank_screener = EqualityScreenerObject('fin_statement_type', equal_to="BANK", description="Bank")
not_bank_screener = EqualityScreenerObject('fin_statement_type', not_equal_to="BANK", description="Bank")

dividend_yield_screener = LimitScreenerObject('dividend_yield', min_value=0.1, description="Dividend Yield")
payout_screener = LimitScreenerObject('payout_ratio', max_value=150, description="Payout Ratio")
price_screener = LimitScreenerObject('price', max_value=40, description="Price")
trailing_pe_screener = LimitScreenerObject('trailing_pe', min_value=0, max_value=30, description="Trailing PE")
forward_pe_screener = LimitScreenerObject('forward_pe', min_value=0, max_value=30, description="Forward PE")
revenue_screener = LimitScreenerObject('revenue', min_value=0, lambda_func= lambda x: growth_total(x)[0], description="Revenue Growth")
operating_cashflow_screener = LimitScreenerObject('operating_cashflow', min_value=0, description="Operating Cashflow")
equity_ratio_screener_not_bank = LimitScreenerObject('equity_ratio', min_value=30, description="Equity Ratio", screener_object=not_bank_screener)
equity_ratio_screener_bank = LimitScreenerObject('equity_ratio', min_value=8, description="Equity Ratio", screener_object=bank_screener)
ebitda_margin_screener = LimitScreenerObject('ebitda_margin', min_value=15, description="EBITDA Margin")
net_income_screener = LimitScreenerObject('net_income', min_value=0, lambda_func=np.min, description="Min Net Income")

screener_list = [
    # dividend_yield_screener,
    payout_screener,
    # price_screener,
    # trailing_pe_screener,
    # forward_pe_screener,
    revenue_screener,
    operating_cashflow_screener,
    equity_ratio_screener_not_bank,
    equity_ratio_screener_bank,
    ebitda_margin_screener,
    net_income_screener
]

screened_contracts = screener.screen(screener_list)

/home/jag/projects/stockMarket/stockMarket/yfinance/_common.py:36: UserWarning: Not all tickers are in calendar and the selected update period indicates that the calendar is up to date. Therefore the calendar will not be updated and the missing tickers will be remain empty. To enforce an update the calendar set update to 'now'
  warnings.warn(
/home/jag/projects/stockMarket/stockMarket/yfinance/_common.py:36: UserWarning: Not all tickers are in calendar and the selected update period indicates that the calendar is up to date. Therefore the calendar will not be updated and the missing tickers will be remain empty. To enforce an update the calendar set update to 'now'
  warnings.warn(


### Perform Screening

In [15]:
import numpy as np
import pandas as pd

from stockMarket.core import Ranking, RankingObject

def rank_equity_ratio(contract):
    equity_ratio = contract.equity_ratio[0]
    if equity_ratio > 30:
        return (equity_ratio, 1)
    if equity_ratio < 10:
        return (equity_ratio, -1)
    return (equity_ratio, 0)

def netto_margin(contract):
    netto_margin = contract.netto_margin[0]
    if 10 < netto_margin < 20:
        return netto_margin, 1
    if netto_margin > 20:
        return netto_margin, 2
    
    return netto_margin, 0

def return_on_assets(contract):
    return_on_assets = contract.return_on_assets[0]
    if 5 < return_on_assets < 10:
        return return_on_assets, 1
    if return_on_assets > 10:
        return return_on_assets, 2
    
    return return_on_assets, 0

def goodwill(contract):
    goodwill = contract.goodwill_ratio[0]
    
    if goodwill > 30:
        return goodwill, -5
    
    return goodwill, 0
    

equity_ratio_ranking = RankingObject(rank_equity_ratio, "Equity Ratio")
netto_margin_ranking = RankingObject(netto_margin, "Netto Margin")
return_on_assets_ranking = RankingObject(return_on_assets, "Return on Assets")
goodwill_ranking = RankingObject(goodwill, "Goodwill")
ranking_list = [
    equity_ratio_ranking,
    netto_margin_ranking,
    return_on_assets_ranking,
    goodwill_ranking,
]

ranking = Ranking(screened_contracts, ranking_list)
ranking.rank()
ranking.ranking.to_csv("ranking_contracts.csv", sep='\t', encoding='utf-8', float_format='%.2f')
ranking.ranking

,Name,Sector,Total Score,Equity Ratio,Equity Ratio Score,Netto Margin,Netto Margin Score,Return on Assets,Return on Assets Score,Goodwill,Goodwill Score
EW,Edwards Lifesciences Corporation,Healthcare,5,70.023515,1,28.275490,2,18.352728,2,20.050976,0
TER,"Teradyne, Inc.",Technology,5,70.011927,1,22.677997,2,20.435576,2,16.448251,0
AMAT,"Applied Materials, Inc.",Technology,5,53.203814,1,25.855112,2,22.311172,2,22.827084,0
REGN,"Regeneron Pharmaceuticals, Inc.",Healthcare,5,77.577915,1,35.639823,2,14.850160,2,0.000000,0
VRTX,Vertex Pharmaceuticals Incorporated,Healthcare,5,76.650194,1,37.197532,2,18.302123,2,7.820193,0
...,...,...,...,...,...,...,...,...,...,...,...
CBOE,"Cboe Global Markets, Inc.",Financial Services,-4,49.512066,1,5.936592,0,3.357670,0,90.116296,-5
CHD,"Church & Dwight Co., Inc.",Consumer Defensive,-4,41.817245,1,7.699606,0,4.959500,0,69.537809,-5
TYL,"Tyler Technologies, Inc.",Technology,-4,55.987957,1,8.876859,0,3.503849,0,94.852859,-5
BK,The Bank of New York Mellon Corporation,Financial Services,-5,9.994072,-1,10.007395,1,0.825217,0,39.689048,-5


### Write Email

In [16]:
from stockMarket.utils import write_email

emails = ["benjaminlantschner@gmail.com", "97gamjak@gmail.com"]
subject = "Stock Market Ranking"
body = ""
attachment = "ranking_contracts.csv"
write_email(emails, subject, body, attachment)